In [2]:
import torch

In [17]:
torch.flatten(a, 1)


tensor([[-0.1646,  0.8678,  0.2779, -0.9932],
        [-0.6510,  0.9322, -1.2898, -0.6936]])

In [40]:
inputs = torch.randn(2,2,2)
print (inputs)

inputs = inputs.reshape( (4,2) )
print (inputs)

masked = inputs[ [1,3] ]

outputs = torch.zeros_like(inputs)
print ( outputs )

    
outputs[1] = inputs[0]



tensor([[[-1.9280,  0.7909],
         [ 1.3147, -2.3144]],

        [[ 0.1071,  0.1162],
         [-0.5505,  1.6712]]])
tensor([[-1.9280,  0.7909],
        [ 1.3147, -2.3144],
        [ 0.1071,  0.1162],
        [-0.5505,  1.6712]])
tensor([[ 1.3147, -2.3144],
        [-0.5505,  1.6712]])
tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])


In [32]:
mask = torch.tensor([[1,0], [0,0]])
mask = (mask == 1)
print ("mask", mask)

inputs = torch.randn(2,2,2)
print ("input", inputs)
values = torch.masked_select(inputs.T, mask)
indices = mask.nonzero()
print ("indices", indices)
print ("values", values)

dest = torch.zeros_like(inputs)
indices = torch.tensor([[0,0], [0,1]])
print ("indices", indices)
print(torch.sparse.FloatTensor(indices.T, values, dest.shape).to_dense())


mask tensor([[ True, False],
        [False, False]])
input tensor([[[ 0.4228, -0.3355],
         [ 0.3166, -1.9283]],

        [[ 0.1507,  0.4296],
         [-0.3167,  0.4062]]])
indices tensor([[0, 0]])
values tensor([ 0.4228, -0.3355])
indices tensor([[0, 0],
        [0, 1]])


RuntimeError: number of dimensions must be sparse_dim (2) + dense_dim (0), but got 3

In [79]:
a = torch.randn(2,2)
z = torch.zeros_like(a)
print ("z", z)
print ("a", a)
b = a <= 0
print ("mask", b)
y = torch.masked_select(a, b)
print ("y", y)
indices = b.nonzero()
print ("indices", indices)
print ( torch.sparse.FloatTensor(indices.T, y, z.shape).to_dense())

z tensor([[0., 0.],
        [0., 0.]])
a tensor([[-0.4875,  2.2268],
        [-1.2256,  0.0623]])
mask tensor([[ True, False],
        [ True, False]])
y tensor([-0.4875, -1.2256])
indices tensor([[0, 0],
        [1, 0]])
tensor([[-0.4875,  0.0000],
        [-1.2256,  0.0000]])


In [49]:
x = torch.randn(2, 2, 2)
z = torch.zeros_like(x)

print ("source", x)
mask = x.ge(0.5)
print( "mask", mask )
y = torch.masked_select(x, mask)
print ("values", y)
ss = (mask == True).nonzero(as_tuple=True)
print ("index", ss)

y.scatter_(0, ss, z)




source tensor([[[ 1.4975, -0.8643],
         [-1.0493,  0.2294]],

        [[-0.9519, -1.1548],
         [-1.2702, -0.8277]]])
mask tensor([[[ True, False],
         [False, False]],

        [[False, False],
         [False, False]]])
values tensor([1.4975])
index (tensor([0]), tensor([0]), tensor([0]))


TypeError: scatter_() received an invalid combination of arguments - got (int, tuple, Tensor), but expected one of:
 * (int dim, Tensor index, Tensor src)
      didn't match because some of the arguments have invalid types: (int, !tuple!, Tensor)
 * (int dim, Tensor index, Tensor src, *, str reduce)
 * (int dim, Tensor index, Number value)
      didn't match because some of the arguments have invalid types: (int, !tuple!, !Tensor!)
 * (int dim, Tensor index, Number value, *, str reduce)


In [9]:
b = torch.squeeze(a, 1)
print (a.shape)
print (b.shape)

torch.Size([2, 256, 1024])
torch.Size([2, 256, 1024])


In [10]:
torch.masked_select(a, mask)

TypeError: masked_select(): argument 'mask' (position 2) must be Tensor, not list